In [1]:
import pandas as pd

import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

import gensim
from gensim import corpora
from gensim import similarities
from gensim import models

from random import randint

import dash
import dash_html_components as html
import dash_core_components as dcc
import matplotlib.pyplot as plt

import base64

df=pd.read_csv('data.csv')

C:\Users\80600\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
model = SentimentIntensityAnalyzer()
model.polarity_scores('this is a good example')

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [3]:
selectlist=['#Davidcameron','#euin','#euout','#INtogether','#NoEu','#TakeControl','#TakeControlDominicFarrell','#votein','#voteleave',
     '#voteout','#voteremain','#yes2eu','#yestoeu','@BrexitWatch','@britinfluence',
     '@eureferendum','@euromove','@Grassroots_out','@JuneExit','@LeaveEUOfficial','@lsebrexitvote',
     '@sayyes2europe','@StrongerIn','@StrongerInPress','@UKandEU','@vote_leave','@Vote_LeaveMedia','@whatukthinks']
stay=['#euin','#INtogether','#votein','#voteremain','#yes2eu','#yestoeu','@BrexitWatch','@britinfluence', '@eureferendum','@lsebrexitvote',
      '@sayyes2europe','@StrongerIn','@StrongerInPress','@UKandEU','@whatukthinks']

In [ ]:
def change(x):
    if x in stay:
        return 'stay'
    else:
        return 'leave'
df['label'] = df['Tag'].map(lambda x: change(x))

def senti(x):
    sentiscore=model.polarity_scores(x)['compound']
    if sentiscore >= 0.1:
        return 'positive'
    elif sentiscore <= -0.1:
        return 'negative'
    else:
        return 'neutral'
df['senti'] = df['Message'].map(lambda x: senti(x))
dfstay=df[df['label']=='stay']
dfleave=df[df['label']=='leave']
#lpp=len(dfleave[dfleave['senti']=='positive'])/(len(dfleave)-len(dfleave[dfleave['senti']=='neutral']))*len(dfleave)/len(df)+len(dfstay[dfstay['senti']=='negative'])/(len(dfstay)-len(dfstay[dfstay['senti']=='neutral']))*len(dfstay)/len(df)
#spp=len(dfstay[dfstay['senti']=='positive'])/(len(dfstay)-len(dfstay[dfstay['senti']=='neutral']))*len(dfstay)/len(df)+len(dfleave[dfleave['senti']=='negative'])/(len(dfleave)-len(dfleave[dfleave['senti']=='neutral']))*len(dfleave)/len(df)

def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

def generate_barchart(dataframe, max_rows=15):
    return dcc.Graph(
        id='barchart',
        figure={
            'data': [
                {'x': dataframe.columns[1:5], 'y': dataframe.iloc[0][1:5], 'type': 'bar', 'name': 'Stay'},
                {'x': dataframe.columns[1:5], 'y': dataframe.iloc[1][1:5], 'type': 'bar', 'name': 'Leave'},
            ],
            'layout': {
                'title': 'Bar Chart Visualization'
            }
        }
    )


selectdict=[]
for a in selectlist:
    selectdict.append({'label':a,'value':a})

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H2('Brexit Poll Reulst'),
    html.Img(id='image', style={'marginLeft':'10%','marginTop':'3%','width':'60%'}),
    
    html.H2('Choose your stay tags'),
    dcc.Dropdown(
        id='stay-dropdown',
        options=selectdict,
        value=['#euin','#INtogether','#votein','#voteremain','#yes2eu','#yestoeu','@BrexitWatch','@britinfluence', '@eureferendum','@lsebrexitvote',
      '@sayyes2europe','@StrongerIn','@StrongerInPress','@UKandEU','@whatukthinks'],
        multi=True
    ),
    html.H3('Your leave tags'),
    dcc.Dropdown(
        id='leave-dropdown',
        options=selectdict,
        value=['@LeaveEUOfficial', '@Vote_LeaveMedia', '#TakeControl', '@euromove', '#voteleave', '#Davidcameron',
               '@Grassroots_out', '#TakeControlDominicFarrell', '#voteout', '#euout', '@JuneExit', '@vote_leave', '#NoEu'],
        multi=True,
        disabled=True
    ),
    html.H2('Data Visualization'),
    html.Div(id='barchart'),
    html.H2('Tags Breakdowns for Stay and Leave Group'),
    
    html.Div(id='piechart_stay'),
    
    html.H2('Tweets Sentiment Analysis'),
#     html.Div(id='piechart1'),
    html.Img(id='image_senti', style={'marginLeft':'10%','marginTop':'3%','width':'60%'}),
    html.H2('Tweets Sentiment Analysis Result'),
    html.Div(id='percentagebysenti'),
    

],style={'marginLeft':'10%','marginTop':'5%','width':'80%'})



@app.callback(
    dash.dependencies.Output('leave-dropdown', 'value'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def updateLeaveBox(value):
    value2=list(set(selectlist).difference(set(value)))
    return value2





@app.callback(
    dash.dependencies.Output('barchart', 'children'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def display_BarChart(value):
    def change(x):
        if x in value:
            return 'stay'
        else:
            return 'leave'
    df['label'] = df['Tag'].map(lambda x: change(x))

    barxlist= [ 'TotalTweets', 'Usercount', 'Retweet', 'TotalLikes'] 
    barystaylist = [df['label'].tolist().count('stay'), len(set(df[df['label']=='stay']['Userid'].tolist())), df[df['label']=='stay']['Retweet'].sum(), df[df['label']=='stay']['Likes'].sum() ]
    baryleavelist = [df['label'].tolist().count('leave'), len(set(df[df['label']=='leave']['Userid'].tolist())),df[df['label']=='leave']['Retweet'].sum(), df[df['label']=='leave']['Likes'].sum() ]
    data = [{'x': barxlist, 'y': barystaylist, 'type': 'bar', 'name': 'Stay'},
            {'x': barxlist, 'y': baryleavelist, 'type': 'bar', 'name': 'Leave'},]

    return dcc.Graph(
        figure={
            'data': data,
            'layout': {
                'title': 'Data Visualization'
            }
        }
    )




@app.callback(
    dash.dependencies.Output('image', 'src'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def update_image_src(value):
    image_path = "poll.png"
    # print the image_path to confirm the selection is as expected
    #print('current image_path = {}'.format(image_path))
    encoded_image = base64.b64encode(open(image_path, 'rb').read())
    return 'data:image/png;base64,{}'.format(encoded_image.decode())

@app.callback(
    dash.dependencies.Output('image_senti', 'src'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def update_image_src(value):
    image_path = "senti.jpeg"
    # print the image_path to confirm the selection is as expected
    #print('current image_path = {}'.format(image_path))
    encoded_image = base64.b64encode(open(image_path, 'rb').read())
    return 'data:image/jpeg;base64,{}'.format(encoded_image.decode())

@app.callback(
    dash.dependencies.Output('piechart_stay', 'children'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def display_PieChart2(value):
    def change(x):
        if x in value:
            return 'stay'
        else:
            return 'leave'
    df['label'] = df['Tag'].map(lambda x: change(x))
    df_stay = df[df.label == 'stay']
    df_leave = df[df.label == 'leave']
    df_stay_count = df_stay.groupby('Tag').count()
    df_leave_count = df_leave.groupby('Tag').count()
    x_stay = df_stay_count.index.tolist()
    x_leave = df_leave_count.index.tolist()
    y_stay =  df_stay_count.iloc[:]['label'][:].tolist()
    y_leave =  df_leave_count.iloc[:]['label'][:].tolist()


    data_stay = [
        {
            'labels': x_stay,
            'values': y_stay,
            'type': 'pie',
        }

    ]
    data_leave = [
        {
            'labels': x_leave,
            'values': y_leave,
            'type': 'pie',
        }

    ]
    return html.Div([
        dcc.Graph(
            figure={
                'data': data_stay,
                'layout': {
                    'margin': {
                        'l': 100,
                        'r': 0,
                        'b': 30,
                        't': 0
                    },
                    'legend': {'x': 0, 'y': 1}
                }
            }
        ),
        dcc.Graph(
            figure={
                'data': data_leave,
                'layout': {
#                     'title': 'Pie Chart Visualization',
                    'margin': {
                        'l': 100,
                        'r': 0,
                        'b': 30,
                        't': 0
                    },
                    'legend': {'x': 0, 'y': 1}
                }
            }
        ),
        
    ])

@app.callback(
    dash.dependencies.Output('percentagebysenti', 'children'),
    [dash.dependencies.Input('stay-dropdown', 'value')])
def sentipercent(value):
#     dfstay=df[df['label']=='stay']
#     dfleave=df[df['label']=='leave']
#     dfstay.to_csv("liststay1.csv", sep='\t')
#     dfleave.to_csv("listleave1.csv", sep='\t')
#     lpp=len(dfleave[dfleave['senti']=='positive'])/(len(dfleave)-len(dfleave[dfleave['senti']=='neutral']))*len(dfleave)/len(df)+len(dfstay[dfstay['senti']=='negative'])/(len(dfstay)-len(dfstay[dfstay['senti']=='neutral']))*len(dfstay)/len(df)
#     spp=len(dfstay[dfstay['senti']=='positive'])/(len(dfstay)-len(dfstay[dfstay['senti']=='neutral']))*len(dfstay)/len(df)+len(dfleave[dfleave['senti']=='negative'])/(len(dfleave)-len(dfleave[dfleave['senti']=='neutral']))*len(dfleave)/len(df)
    
    df_stay = df[df.label == 'stay' ]
    df_leave = df[df.label == 'leave' ] 
    def changeStaySenti(x):
        if x == "negative":
            return 'leave'
        else:
            return 'stay'
    def changeLeaveSenti(x):
        if x == "negative":
            return 'stay'
        else:
            return 'leave'
    df_stay['label'] = df_stay['senti'].map(lambda x: changeStaySenti(x))
    df_leave['label'] = df_leave['senti'].map(lambda x: changeLeaveSenti(x))
    df_senti  = pd.concat([df_stay, df_leave])
    df_senti_stay = df_senti[df_senti['label']=="stay"]
    df_senti_leave = df_senti[df_senti['label']=="leave"]
    df_senti_stay.to_csv("senti_stay2.csv")
    df_senti_leave.to_csv("senti_leave2.csv")
    
    stay_senti_counts = len(df_senti_stay)
    leave_senti_counts = len(df_senti_leave)
    
    
    data = [
        {
            'labels': ['Stay', 'Leave'],
#             'values': [ ((df['label'].tolist().count('stay')/len(df[['label']])) * 100),((df['label'].tolist().count('leave')/len(df[['label']])) * 100)],
            'values':   [stay_senti_counts, leave_senti_counts],
            'type': 'pie',
        },
    ]

    return html.Div([
        dcc.Graph(
            figure={
                'data': data,
                'layout': {
#                     'title': 'Pie Chart Visualization',
                    'margin': {
                        'l': 30,
                        'r': 0,
                        'b': 30,
                        't': 0
                    },
                'legend': {'x': 0, 'y': 1},
                "marker": {'colors': ['#e6f2ff', '#99ccff']},
                }
            }
            
            
        )
    ])

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2021 15:19:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 15:20:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 15:20:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 15:20:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 15:20:03] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\80600\Anaconda3\lib\site-packages\ipykernel_launcher.py:246: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [28/Jul/2021 15:20:03] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\80600\Anaconda3\lib\site-packages\ipykernel_launcher.py:247: SettingWithCopyWarning:


A value is trying to be s

In [ ]:
import webbrowser
url = 'stay.html'
webbrowser.open(url, new=2)

In [ ]:
import webbrowser
url = 'leave.html'
webbrowser.open(url, new=2)